In [1]:
%pip install trl

  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached xxhash-3.5.0-cp313-cp313-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached yarl-1.18.3-cp313-cp313-win_amd64.whl.metadata (71 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached rich-13.9.4-py3-none-any.whl (242 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
Using cached markdown_it_py-

In [2]:
%pip install transformers

In [3]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [22]:
%pip install pandas

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

checkpoint = "HuggingFaceTB/SmolLM-135M-Instruct"
device = "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do
# model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

In [ ]:
ds = load_dataset("medalpaca/medical_meadow_medical_flashcards")

In [10]:
pandas_data = pd.DataFrame(ds['train'])
pandas_data = pandas_data[:10]
# pandas_data.dropna(inplace=True)
ds = Dataset.from_pandas(pandas_data)

In [19]:
def tokenize_function(examples):
    examples["text"] = tokenizer.apply_chat_template([{"role": "user", "content": examples["input"]}, {"role": "assistant", "content": examples["output"]}], tokenize=False)
    return examples
ds = ds.map(tokenize_function)

Map: 100%|██████████| 10/10 [00:00<00:00, 694.35 examples/s]


In [20]:
sft_config = SFTConfig(
    output_dir="./sft_output",
    num_train_epochs=10,
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=100,  # Frequency of logging training metrics
    use_mps_device= True if device == "mps" else False,
    # hub_model_id="argilla/SmolLM2-360M-synthetic-concise-reasoning2",  # Set a unique name for your model
    push_to_hub=False,
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds,
    # tokenizer=tokenizer,
)
trainer.train()

Truncating train dataset: 100%|██████████| 10/10 [00:00<00:00, 795.51 examples/s]


Step,Training Loss


TrainOutput(global_step=30, training_loss=0.14918750127156574, metrics={'train_runtime': 94.4534, 'train_samples_per_second': 1.059, 'train_steps_per_second': 0.318, 'total_flos': 3476676335616.0, 'train_loss': 0.14918750127156574})

In [21]:
# prompt = "What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?"

prompt = "Low REM sleep latency"

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
pipe(prompt, max_new_tokens=30)

Device set to use cpu


[{'generated_text': 'Low REM sleep latency and duration\n* Difficulty falling asleep or staying asleep\n* Waking up frequently\n* Irritability and mood changes\n* Decreased productivity and memory'}]